TO INSTALL

pip install torch torchvision torchaudio

pip install pywavelets

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pywt
import numpy as np

In [5]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def to_wavelet_coeffs(data, wavelet='db1', mode='symmetric'):
    coeffs = pywt.wavedec2(data, wavelet=wavelet, mode=mode)
    # Flatten and concatenate coefficients for CNN input
    coeffs_flat = [coeffs[0].flatten()]  # Flatten the first array (approximation)
    for detail_coeff in coeffs[1:]:
        # Flatten each part of the tuple and then concatenate
        coeffs_flat.extend([c.flatten() for c in detail_coeff])
    return np.concatenate(coeffs_flat)

In [6]:
# Example usage
if __name__ == "__main__":
    model = SimpleCNN()

    # Example data (replace with your actual data)
    data = np.random.rand(28, 28)
    wavelet_coeffs = to_wavelet_coeffs(data)

    # Convert to PyTorch tensor and add batch dimension
    input_tensor = torch.tensor(wavelet_coeffs).float().unsqueeze(0)

    # Forward pass through the model
    output = model(input_tensor)
    print(output)

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 799]